In [1]:
import pandas as pd
import numpy as np
import csv
from pathlib import Path 
import re
import string

In [2]:
df= pd.read_csv(r'/Users/lolitalolita/Downloads/recipes.csv')

In [3]:
df

,RecipeId,Name,RecipeCategory,Calories,CholesterolContent,CarbohydrateContent,SugarContent,ProteinContent,RecipeServings,HighScore
0,46085,Crock Pot Baked Potato Soup,One Dish Meal,699.8,137.3,46.1,1.4,20.9,6,1
1,93832,Frittata Di Spaghetti (spaghetti Frittata),Breakfast,297.1,191.8,11.7,0.7,12.2,8,1
2,36034,Berries With Italian Cream,Dessert,131.9,23.3,10.3,4.4,9.1,6,0
3,329988,Pork Tenderloin Medallions With Fresh Figs,< 15 Mins,203.0,74.8,1.5,0.6,23.3,4,1
4,59886,Kaseropita (Tiropita Using Kaseri Cheese),Savory Pies,261.6,103.6,20.9,0.2,6.7,15,1
...,...,...,...,...,...,...,...,...,...,...
43087,22945,Yummy Meatloaf,Meat,281.5,111.3,13.2,11.1,25.3,6,1
43088,110033,Slow Cooker Shanks,Lamb/Sheep,697.2,242.1,3.8,1.9,72.9,4,0
43089,240945,Sin-Free Chocolate Chews,Drop Cookies,94.5,0.0,22.7,19.1,2.0,14,1
43090,164157,Montreal Salsa Chicken,Chicken Breast,207.1,72.6,20.1,12.7,24.9,4,1


In [4]:
# Get first 5000 recipes 
df=df[:5000]


In [5]:
# Get calories and content per one serving
df_per_serving=df[['Calories','CholesterolContent','CarbohydrateContent','SugarContent','ProteinContent']].div(df.RecipeServings, axis=0) 

In [6]:
df_per_serving

,Calories,CholesterolContent,CarbohydrateContent,SugarContent,ProteinContent
0,116.633333,22.883333,7.683333,0.233333,3.483333
1,37.137500,23.975000,1.462500,0.087500,1.525000
2,21.983333,3.883333,1.716667,0.733333,1.516667
3,50.750000,18.700000,0.375000,0.150000,5.825000
4,17.440000,6.906667,1.393333,0.013333,0.446667
...,...,...,...,...,...
4995,27.275000,1.900000,1.300000,0.250000,0.400000
4996,2.191667,0.000000,0.483333,0.191667,0.100000
4997,68.512500,6.612500,12.987500,6.887500,1.337500
4998,107.700000,0.000000,15.333333,4.233333,2.400000


In [8]:
# Import PuLP modeler functions
from pulp import *

In [9]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("Minimal_cholesterol_intake", LpMinimize)

/Users/lolitalolita/opt/anaconda3/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [10]:
Name_dict = list(df['Name'])

Calories_dict = dict(zip(Name_dict, df_per_serving['Calories']))
CholesterolContent_dict = dict(zip(Name_dict , df_per_serving['CholesterolContent']))
CarbohydrateContent_dict = dict(zip(Name_dict , df_per_serving['CarbohydrateContent']))
SugarContent_dict = dict(zip(Name_dict, df_per_serving['SugarContent']))
ProteinContent_dict = dict(zip(Name_dict, df_per_serving['ProteinContent']))

In [11]:
len(Name_dict)

5000

In [12]:
# A dictionary called 'ingredient_vars' is created to contain the referenced Variables
recipe_name_vars = LpVariable.dicts("R", Name_dict, 0)

In [13]:
# The objective function is added to 'prob' first
prob += (
    lpSum([CholesterolContent_dict[i] * recipe_name_vars[i] for i in Name_dict]),
    "Total sugar content per dish",)

In [14]:
# The constraints are added to 'prob'
 

prob += (
    lpSum([Calories_dict[i] * recipe_name_vars[i] for i in Name_dict]) <= 2000.0,
    "Calories_dict_Requirement_max",
)
prob += (
    lpSum([Calories_dict[i] * recipe_name_vars[i] for i in Name_dict]) >= 1500.0,
    "Calories_dict_Requirement_min",
)


prob += (
    lpSum([SugarContent_dict[i] * recipe_name_vars[i] for i in Name_dict]) <= 30.0,
    "SugarContent_Requirement_max",
)
prob += (
    lpSum([SugarContent_dict[i] * recipe_name_vars[i] for i in Name_dict]) >= 5.0,
    "SugarContent_Requirement_min",
)

prob += (
    lpSum([CarbohydrateContent_dict[i] * recipe_name_vars[i] for i in Name_dict]) <= 150.0,
    "CarbohydrateContent_Requirement_max",
)

prob += (
    lpSum([CarbohydrateContent_dict[i] * recipe_name_vars[i] for i in Name_dict]) >= 120.0,
    "CarbohydrateContent_Requirement_min",
)

prob += (
    lpSum([ProteinContent_dict[i] * recipe_name_vars[i] for i in Name_dict]) <= 112.0,
    "ProteinContent_Requirement_max",
)
prob += (
    lpSum([ProteinContent_dict[i] * recipe_name_vars[i] for i in Name_dict]) >= 70.0,
    "ProteinContent_Requirement_min",
)

In [15]:
# The problem data is written to an .lp file
prob.writeLP("RecipeModel.lp")

[R_"All_in_the_Kitchen"_Chili,
 R_"Immoral"_Sandwich_Filling_(Loose_Meat),
 R_"Jiffy"_Roasted_Corn_and_Jalapeno_Cornbread,
 R_"Smells_Like_Sunday"_Chicken_Fricassee_With_Meatballs,
 R_&quot;_Almost_&quot;_Dairy_Queen_Blizzard_Ice_Cream_Cake,
 R_&quot;_Atkins_Friendly&quot;_Cheesecake,
 R_&quot;_Drunken&quot;_Pork_Chops,
 R_&quot;_No_Harm_Eggplant_Parm&quot;_Ww,
 R_&quot;_Tastes_Like_Mounds_Bar_&quot;__Kahlua_Cake,
 R_&quot;fooled_Me&quot;_Flourless_Peanut_Butter_Cookies,
 R_&quot;sannich&quot;_in_Soup,
 R_'Get_Up_&_Go'_Bars,
 R_(Crock_Pot)_Chicken_Chili,
 R_(Fake)_Crab_Stroganoff,
 R_*s.w.o.c.*_Pie_Cookies,
 R_100%_Whole_Wheat_Bread,_Plain_and_Simple_(No_Knead),
 R_15_Minute_Chili_Cheeseburger_Skillet,
 R_24_Hour_Copper_Pennies,
 R_24_Hour_Lettuce_Salad,
 R_2_Minute_Noodle_Corn_Pancakes,
 R_3_Bean_Salad,
 R_3_Cheese_Manicotti,
 R_3_Dragon’s_Szechuan_Steak,
 R_3_Layered_Mexican_Dip,
 R_3_Minute_Bread_Pudding_in_a_Mug,
 R_3_Peppercorn_Hazelnut_Soup,
 R_3_Spice_Fuss_Free_Fish_Fillet_Curry

In [16]:
# The problem is solved using PuLP's choice of Solver
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/lolitalolita/opt/anaconda3/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/j9/7y5q0v6x4p3f2fdr5f5ffl0m0000gn/T/a364f59333e64753942fce4627ccec45-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/j9/7y5q0v6x4p3f2fdr5f5ffl0m0000gn/T/a364f59333e64753942fce4627ccec45-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 43200 RHS
At line 43209 BOUNDS
At line 43210 ENDATA
Problem MODEL has 8 rows, 4941 columns and 39244 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 7 (-1) rows, 4924 (-17) columns and 34286 (-4958) elements
0  Obj 0 Primal inf 242.83033 (3)
4  Obj 1.9230129e-16
Optimal - objective value 0
After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 0 - 4 iterations time 0.012, Presolve 0.01
Opti

1

In [17]:
# The status of thesolution is printed to the screen
print("Status:", LpStatus[prob.status])

Status: Optimal


In [18]:
# Each of the variables is printed with it's resolved optimum value
    
for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

R_3_Spice_Fuss_Free_Fish_Fillet_Curry = 24.966443
R_Ann's_Easy_Posole = 19.328859
R_Baileys_Mint_Chocolate_Martini = 22.613268
R_Jello_Fruit_Bars = 222.81879


In [19]:
# The optimised objective function value is printed to the screen
print("Minimal cholesterol intake = ", value(prob.objective))

Minimal cholesterol intake =  0.0
